In [2]:
import cv2
import os
import numpy as np
from IPython.display import clear_output

path="/content/drive/MyDrive"

path1="/content/drive/MyDrive/Copy of fake/"
files_fake = os.listdir(path1)

path2="/content/drive/MyDrive/Copy of real/"
files_real=os.listdir(path2)
files_real.remove("desktop.ini")
files_fake.remove("desktop.ini")
print (len(files_fake),len(files_real))

1710 1710


# New Section

In [3]:
#@title
features=["brightness,contrast,pattern,color_difference,edge_density,fractional_dimension,b,g,r,var_b,var_g,var_r,magnitude_spectrum,hist_b,hist_g,hist_r,label\n"]
# Load the image
def extract(image):

    img = cv2.imread(image)
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    magnitude_spectrum = np.sum(magnitude_spectrum)
    b, g, r = cv2.split(img)

    hist_b = cv2.calcHist([img], [0], None, [256], [0, 256])
    hist_b = np.max(hist_b)
    hist_g = cv2.calcHist([img], [1], None, [256], [0, 256])
    hist_g = np.max(hist_g)
    hist_r = cv2.calcHist([img], [2], None, [256], [0, 256])
    hist_r = np.max(hist_r)

    # Calculate mean intensity values for each channel
    mean_b = np.mean(b)
    mean_g = np.mean(g)
    mean_r = np.mean(r)

    # Calculate squared difference for each pixel in each channel
    diff_b = (b - mean_b) ** 2
    diff_g = (g - mean_g) ** 2
    diff_r = (r - mean_r) ** 2

    # Calculate variance for each channel
    var_b = np.mean(diff_b)
    var_g = np.mean(diff_g)
    var_r = np.mean(diff_r)
    b, g, r = cv2.mean(img)[:3]
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Calculate brightness
    brightness = np.mean(gray)

    # Calculate contrast
    stddev = np.std(gray)
    contrast = stddev / brightness

    # Calculate texture pattern using a Gabor filter
    ksize = 31 # kernel size
    theta = np.pi/4 # orientation of filter
    sigma = 5 # standard deviation of Gaussian envelope
    lambda_ = 10 # wavelength of sinusoidal factor
    gamma = 0.5 # spatial aspect ratio
    psi = 0 # phase offset
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambda_, gamma, psi)
    filtered = cv2.filter2D(gray, cv2.CV_8UC3, kernel)
    texture_pattern = np.mean(filtered)

    # Calculate color difference between masks
    mask1 = cv2.inRange(img, (0, 0, 0), (50, 50, 50))
    mask2 = cv2.inRange(img, (200, 200, 200), (255, 255, 255))
    color_diff = np.mean(cv2.absdiff(mask1, mask2))

    # Other features to distinguish AI-generated and natural images:
    # - Edge density: measure the number of edges in the image
    # - Fractal dimension: measure the complexity of the image at different scales
    # - Color histogram: compare the distribution of colors in the image with those of known datasets of natural and AI-generated images

    # Edge density
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges) / (img.shape[0] * img.shape[1])

    # Fractal dimension
    def fractal_dimension(Z, threshold=0.9):
        # Only for 2d image
        assert(len(Z.shape) == 2)

        def boxcount(Z, k):
            S = np.add.reduceat(
                np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                                    np.arange(0, Z.shape[1], k), axis=1)
            return np.count_nonzero(S)

        # Transform Z into a binary array
        Z = (Z < threshold)

        # Minimal dimension of image
        p = min(Z.shape)

        # Greatest power of 2 less than or equal to p
        n = 2**np.floor(np.log(p)/np.log(2))

        # Extract the exponent
        n = int(np.log(n)/np.log(2))

        # Build successive box sizes (from 2**n down to 2**1)
        sizes = 2**np.arange(n, 1, -1)

        # Actual box counting with decreasing size
        counts = []
        for size in sizes:
            counts.append(boxcount(Z, size))
        sizes = np.array(sizes) + 1e-10
        counts = np.array(counts) + 1e-10
        # Fit the successive log(sizes) with log (counts)
        coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
        return -coeffs[0]


    return (str(brightness),str(contrast),str(texture_pattern),str(color_diff),str(edge_density),
            str(fractal_dimension(gray)),str(b),str(g),str(r),str(var_b),str(var_g),str(var_r),str(magnitude_spectrum),str(hist_b),str(hist_g),str(hist_r))
for j,i in enumerate(files_real):
    print (j)
    try: features_i=(extract(path2+i))
    except : continue
    new_labels=i.split("_")
    try :
        if new_labels[3] in ("sea5","see","sea"):
            temp = "sea"
        elif new_labels[3] in ("forest","jungle","junlge"):
            temp = "jungle"
        else :
            temp = "mountain"
        new_label=new_labels[1]+"_"+temp.lower()
        new_label=new_label.lower()

    except :
        new_labels=i.split("-")
        if new_labels[3] in ("sea5","see","sea"):
            temp = "sea"
        elif new_labels[3] in ("forest","jungle","junlge"):
            temp = "jungle"
        else :
            temp = "mountain"
        #print (new_labels)

        new_label=new_labels[1]+"_"+temp.lower()
        new_label=new_label.lower()

    features.append(",".join(features_i)+","+new_label+"\n")
    clear_output()
for j,i in enumerate(files_fake):
    print (j)
    try:features_i=(extract(path1+i))
    except : continue
    new_labels=i.split("_")
    try :
        if new_labels[3] in ("sea5","see","sea"):
            temp = "sea"
        elif new_labels[3] in ("forest","jungle","junlge"):
            temp = "jungle"
        else :
            temp = "mountain"
        new_label=new_labels[1]+"_"+temp.lower()
        new_label=new_label.lower()

    except :
        new_labels=i.split("-")
        if new_labels[3] in ("sea5","see","sea"):
            temp = "sea"
        elif new_labels[3] in ("forest","jungle","junlge"):
            temp = "jungle"
        else :
            temp = "mountain"
        #print (new_labels)

        new_label=new_labels[1]+"_"+temp.lower()
        new_label=new_label.lower()


    features.append(",".join(features_i)+","+new_label+"\n")
    clear_output()
    #clear_output()



In [4]:
print (len(features))

3418


In [6]:
with open (path+"/dataset2.csv","w") as f:
  f.writelines(features)

In [ ]:
print (len(features[0]))

149


In [8]:
print (features[1].split(","))

['99.702134375', '0.41656957163264496', '175.309871875', '24.562875', '88.4031875', '0.5364787773742308', '59.150189583333336', '108.06291354166667', '98.76963125', '1302.1270160057243', '1896.2165241779414', '1806.4314052390234', '343937941.5465197', '53478.0', '8881.0', '9127.0', 'real_jungle\n']
